# Actions e Forms
Tecnologias utilizadas na implementação do sistema de compras públicas, o Compras Brasil.

	

In [ ]:
import { arquivoRepository, arquivoStorage, mensagemRepository, processoRepository } from "@/libs";
import { ProcessoPk, type Processo } from "@/libs/processos";
import { ActionError, defineAction } from "astro:actions";
import { z } from "astro:schema";

In [ ]:



const monetaryValueSchema = z.string().transform((value, ctx) => {
	const cleanedValue = value.replace(/[R$\s]/g, '').replace(/\./g, '').replace(',', '.');

	const numValue = Number.parseFloat(cleanedValue);

	if (Number.isNaN(numValue)) {
		ctx.addIssue({
			code: z.ZodIssueCode.custom,
			message: "Not a number"
		})

		return z.NEVER
	}

	return numValue;
});

const fractionalQuantitySchema = z.string().transform((value, ctx) => {
	const cleanedValue = value.replace(/\./g, '').replace(',', '.');

	const numValue = Number.parseFloat(cleanedValue);

	if (Number.isNaN(numValue)) {
		ctx.addIssue({
			code: z.ZodIssueCode.custom,
			message: "Not a number"
		})

		return z.NEVER
	}

	return numValue;
})

const checkboxSchema = z.union([z.null(), z.literal("on")]).transform(value => value === "on")

const emptyableStringSchema = z.preprocess(value => {
	if (value === null) {
		return ""
	}

	return value
}, z.coerce.string())

export const processo = {
	editar_informacoes: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			modalidade: z.literal("pregao_eletronico"),
			descricao: emptyableStringSchema,
			numero: emptyableStringSchema,
			amparo_legal: emptyableStringSchema,
			permite_registro_precos: z.boolean(),
			permite_carona: z.boolean(),
			prazo_validade_registro_preco: z.coerce.date(),
			data_inicio_propostas: z.coerce.date(),
			data_limite_propostas: z.coerce.date(),
			data_abertura_propostas: z.coerce.date(),
			data_limite_impugnacoes: z.coerce.date(),
			data_limite_esclarecimentos: z.coerce.date(),
			data_inicio_sessao: z.coerce.date(),
			agente_contratacao: z.coerce.number(),
			equipe_contratacao: z.coerce.number(),
			telefone_contato: emptyableStringSchema,
			email_contato: emptyableStringSchema,
			convenio_transferencia: emptyableStringSchema,
			origem_recurso: emptyableStringSchema,
			numero_convenio_ano: emptyableStringSchema,
			"processo_pregao_eletronico.criterio_julgamento": emptyableStringSchema,
			"processo_pregao_eletronico.modo_disputa": z.union([z.literal("aberta_fechada"), z.literal("aberta")]),
			"processo_pregao_eletronico.valor_intervalo_lances": monetaryValueSchema,
			"processo_pregao_eletronico.ordem_fases": z.union([z.literal("classificacao_disputa_habilitacao"), z.literal("habilitacao_disputa_classificacao")]),
			"processo_pregao_eletronico.tipo_lances": z.union([z.literal("lote"), z.literal("lote_item")]),
			"processo_pregao_eletronico.inexequibilidade_automatica": checkboxSchema,
		}),
		handler(input) {
			return processoRepository.update(input.processo_pk, {
				descricao: input.descricao,
				numero: input.numero,
				modalidade: input.modalidade,
				amparo_legal: input.amparo_legal,
				permite_registro_precos: input.permite_registro_precos,
				permite_carona: input.permite_carona,
				prazo_validade_registro_preco: input.prazo_validade_registro_preco,
				data_inicio_propostas: input.data_inicio_propostas,
				data_limite_propostas: input.data_limite_propostas,
				data_abertura_propostas: input.data_abertura_propostas,
				data_limite_impugnacoes: input.data_limite_impugnacoes,
				data_limite_esclarecimentos: input.data_limite_esclarecimentos,
				data_inicio_sessao: input.data_inicio_sessao,
				telefone_contato: input.telefone_contato,
				email_contato: input.email_contato,
				convenio_transferencia: input.convenio_transferencia,
				origem_recurso: input.origem_recurso,
				numero_convenio_ano: input.numero_convenio_ano,
				pregao_eletronico: {
					criterio_julgamento: input["processo_pregao_eletronico.criterio_julgamento"],
					modo_disputa: input["processo_pregao_eletronico.modo_disputa"],
					valor_intervalo_lances: input["processo_pregao_eletronico.valor_intervalo_lances"],
					ordem_fases: input["processo_pregao_eletronico.ordem_fases"],
					tipo_lances: input["processo_pregao_eletronico.tipo_lances"],
					inexequibilidade_automatica: input["processo_pregao_eletronico.inexequibilidade_automatica"],
				}
			})
		}
	}),

	editar_documento: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			arquivo_codigo: z.string(),
			action: z.literal("delete").optional(),
			tipo: z.union([z.literal("processo"), z.literal("complementar")]),
			"arquivo.nome": z.string(),
		}),
		async handler(input) {
			switch (input.action) {
				case "delete":
					return arquivoRepository.delete({
						codigo: input.arquivo_codigo
					})
				default:
					return processoRepository.updateProcessoDocumento({
						processo_codigo: input.processo_pk.codigo,
						processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
						processo_tenant_id: input.processo_pk.tenant_id,
						arquivo_codigo: input.arquivo_codigo
					}, {
						tipo: input.tipo,
						arquivo: {
							nome: input["arquivo.nome"]
						}
					})
			}
		}
	}),

	novo_documento: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			documento: z.instanceof(File)
		}),
		async handler(input) {
			const { codigo } = await arquivoStorage.create({
				file: input.documento,
				path: `tenants/${input.processo_pk.tenant_id}/organizacoes/${input.processo_pk.organizacao_codigo}/processos/${input.processo_pk.codigo}/${input.documento.name}`,
			});

			const arquivo = await arquivoRepository.create({
				codigo,
				nome: input.documento.name
			});

			await processoRepository.createProcessoDocumento(
				[{
					processo_tenant_id: input.processo_pk.tenant_id,
					processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
					processo_codigo: input.processo_pk.codigo,
					arquivo_codigo: arquivo.codigo
				}],
			);
		}
	}),
	editar_informacoes_lote: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			codigo: z.coerce.number(),
			descricao: emptyableStringSchema,
			exclusivo_me_epp: checkboxSchema,
			beneficio_regional: checkboxSchema,
		}),
		handler(input) {
			return processoRepository.updateLote({
				processo_tenant_id: input.processo_pk.tenant_id,
				processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
				processo_codigo: input.processo_pk.codigo,
				codigo: input.codigo,
			}, {
				descricao: input.descricao,
				exclusivo_me_epp: input.exclusivo_me_epp,
				beneficio_regional: input.beneficio_regional,
			})
		}
	}),
	editar_informacoes_lote_item: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			lote_codigo: z.coerce.number(),
			codigo: z.coerce.number(),
			descricao: z.string(),
			quantidade: fractionalQuantitySchema,
			unidade_medida: z.string(),
			valor_unitario: monetaryValueSchema
		}),
		handler(input) {
			return processoRepository.updateLoteItem({
				processo_tenant_id: input.processo_pk.tenant_id,
				processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
				processo_codigo: input.processo_pk.codigo,
				lote_codigo: input.lote_codigo,
				codigo: input.codigo,
			}, {
				descricao: input.descricao,
				quantidade: input.quantidade,
				unidade_medida: input.unidade_medida,
				valor_unitario: input.valor_unitario,
			})
		}
	}),
	controle_andamento: defineAction({
		accept: "form",
		input: z.object({
			"pregao_eletronico.etapa": z.union([
				z.literal('preparatoria'),
				z.literal('divulgacao'),
				z.literal('analise'),
				z.literal('disputa'),
				z.literal('julgamento'),
				z.literal('habilitacao'),
				z.literal('recursal'),
				z.literal('homologacao'),
				z.literal('finalizado'),
			]),
			processo_pk: ProcessoPk
		}),
		async handler(input) {
			const processo = await processoRepository.update(input.processo_pk, {
				pregao_eletronico: {
					etapa: input["pregao_eletronico.etapa"]
				}
			})

			return processo
		}
	}),
	novo_lote_item: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			lote_codigo: z.coerce.number()
		}),
		async handler(input) {
			return await processoRepository.createLoteItem({
				processo_tenant_id: input.processo_pk.tenant_id,
				processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
				processo_codigo: input.processo_pk.codigo,
				lote_codigo: input.lote_codigo
			})
		}
	}),
	novo_lote: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk
		}),
		async handler(input) {
			return await processoRepository.createLote({
				processo_tenant_id: input.processo_pk.tenant_id,
				processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
				processo_codigo: input.processo_pk.codigo,
			})
		},
	}),
	remover_lote: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			codigo: z.coerce.number()
		}),
		async handler(input) {
			return await processoRepository.deleteLote({
				processo_tenant_id: input.processo_pk.tenant_id,
				processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
				processo_codigo: input.processo_pk.codigo,
				codigo: input.codigo
			})
		},
	}),
	remover_lote_item: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			lote_codigo: z.coerce.number(),
			codigo: z.coerce.number()
		}),
		async handler(input) {
			return await processoRepository.deleteLoteItem({
				processo_tenant_id: input.processo_pk.tenant_id,
				processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
				processo_codigo: input.processo_pk.codigo,
				lote_codigo: input.lote_codigo,
				codigo: input.codigo
			})
		},
	}),
	novo: defineAction({
		accept: "form",
		input: z
			.object({
				edital: z.instanceof(File),
				modalidade: z.literal("pregao_eletronico"),
				template_processo_codigo: z.coerce.number(),
			})
			.partial(),
		async handler(input, context) {
			if (!context.locals.user) {
				throw new ActionError({
					code: "UNAUTHORIZED",
					message: "User must be logged in.",
				});
			}

			if (!context.locals.session?.role) {
				throw new ActionError({
					code: "UNAUTHORIZED",
					message: "Role must be set.",
				});
			}

			const { role } = context.locals.session;

			const { tenant_id, organizacao_codigo } = role;

			let processo: Processo;

			if (input.edital && input.edital.size > 0) {
				processo = await processoRepository.create({
					tenant_id,
					organizacao_codigo,
					modalidade: "pregao_eletronico",
				});

				const { codigo } = await arquivoStorage.create({
					file: input.edital,
					path: `tenants/${tenant_id}/organizacoes/${organizacao_codigo}/processos/${processo.codigo}/${input.edital.name}`,
				});

				const arquivo = await arquivoRepository.create({
					codigo,
					nome: input.edital.name
				});

				await processoRepository.createProcessoDocumento(
					[{
						processo_tenant_id: processo.tenant_id,
						processo_organizacao_codigo: processo.organizacao_codigo,
						processo_codigo: processo.codigo,
						tipo: "processo",
						arquivo_codigo: arquivo.codigo
					}],
				);
			} else if (input.modalidade) {
				processo = await processoRepository.create({
					tenant_id,
					organizacao_codigo,
					modalidade: input.modalidade,
				});
			} else if (input.template_processo_codigo) {
				processo = await processoRepository.create({
					tenant_id,
					organizacao_codigo,
					template_processo_codigo: input.template_processo_codigo,
				});
			} else {
				throw new Error();
			}

			return {
				type: "redirect",
				to: `/processos/${processo.tenant_id}_${processo.organizacao_codigo}_${processo.codigo}`,
			};
		},
	}),

	nova_mensagem: defineAction({
		accept: "form",
		input: z.object({
			processo_pk: ProcessoPk,
			tipo: z.union([
				z.literal('processo'),
				z.literal('lote'),
				z.literal('lance'),
				z.literal('solicitacao'),
				z.literal('proposta'),
				z.literal('resposta_mensagem'),
				z.literal('habilitacao')
			]),
			lote_codigo: z.coerce.number().int().optional(),
			tenant_id: z.string(),
			organizacao_codigo: z.coerce.number().int(),
			funcionario_codigo: z.coerce.number().int().optional(),
			conteudo: z.string()
		}),
		async handler(input) {
			return mensagemRepository.create({
				processo_tenant_id: input.processo_pk.tenant_id,
				processo_organizacao_codigo: input.processo_pk.organizacao_codigo,
				processo_codigo: input.processo_pk.codigo,
				tipo: input.tipo,
				tenant_id: input.tenant_id,
				organizacao_codigo: input.organizacao_codigo,
				funcionario_codigo: input.funcionario_codigo ?? null,
				conteudo: input.conteudo,
				mensagem_lote: input.lote_codigo !== undefined ? { lote_codigo: input.lote_codigo } : undefined
			})
		},
	})
};
